Importing libaries

In [3]:
# Importing required libraries for web scraping, data handling, and email sending
from bs4 import BeautifulSoup  # Web Scraping library for pulling the data out of HTML and XML files
import requests  # Allows sending HTTP requests using Python
import smtplib  # Allows sending emails
import time  # Provides various time-related functions
import datetime  # Provides classes for manipulating dates and times
import pandas as pd  # Provides data structures and data analysis tools

Pulling data from amazon
How: Inspect the html of the page and find the element that contains the data we want (Ctrl + Shift + C) allows to select relevant element

In [4]:
# URL of the Amazon product page to scrape
URL = 'https://www.amazon.co.uk/Playstation-711719577157-PlayStation-Console-Slim/dp/B0CM9VHGY7/ref=sr_1_3?crid=3GOR693VEO69U&dib=eyJ2IjoiMSJ9.24sD8SdnVv-3aB5oHBS0Xg-wNRbkkttjoPJRofu3IBpRHCO0w-06laB77gEy8WiYVhqRP-fVdGes6kC5ZPVABiHVzJCQEFTxrky9NW-blNoVatZ1pPMUCOvgPanUPGxA6jbo63_7GByqIA0MAC75i5_MJ7c22zFi2vk5wsJ6ogGKqXL6zsUjY9ctD3TVpADH1ZcCMTEd4COQJ95Q0AVAQ8R_AqX3BEnxttt_eboVgGc.niIIzjtkhGZjZU7E8TDlZtG88NgF1zJhglL5FUjfQhw&dib_tag=se&keywords=playstation&qid=1716120758&sprefix=playstation%2Caps%2C778&sr=8-3&th=1'

# Headers to mimic a real browser for the HTTP request
headers = { 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0", 
    "Accept-Encoding":"gzip, deflate", 
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
    "DNT":"1",
    "Connection":"close", 
    "Upgrade-Insecure-Requests":"1"
}

# Send a GET request to the URL with the specified headers
page = requests.get(URL, headers=headers)

# Parse the HTML content of the page using BeautifulSoup
soup1 = BeautifulSoup(page.content, "html.parser")

# Prettify the HTML content for better readability
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

In [6]:
#Tested with other amazon domains such as .com and it was unable to fetch price so may need to loop through the price block htmls
#Sometimes price and monthly_sales are not available due to item being sold out or not selling enough units

title = None
try:
    title_span = soup2.find("span", {"id": "productTitle"})
    if title_span:
        title = title_span.text.strip()
except Exception as e:
    print(f"Error occurred while extracting title: {e}")

price = None
try:
    price_span = soup2.find("span", {"class": "aok-offscreen"})
    if price_span:
        price = price_span.text.strip()
except Exception as e:
    print(f"Error occurred while extracting price: {e}")

rating = None
try:
    rating_span = soup2.find("span", {"class": "a-icon-alt"})
    if rating_span:
        rating = rating_span.text.strip()
except Exception as e:
    print(f"Error occurred while extracting rating: {e}")

monthly_sales = None
try:
    monthly_sales_span = soup2.find("span", {"id": "social-proofing-faceout-title-tk_bought"})
    if monthly_sales_span:
        monthly_sales = monthly_sales_span.text.strip()
except Exception as e:
    print(f"Error occurred while extracting monthly sales: {e}")

print(title)
print(price)
print(monthly_sales)
print(rating)
#Amazon html for priceblock changed so found code on scarping dog 

PlayStation 5 Console (Slim)
£478.50
1K+ bought in past month
4.7 out of 5 stars


In [1]:
import datetime 

today = datetime.datetime.now()
print(today.strftime("%Y-%m-%d"))

2024-05-19


In [7]:
import csv

header = ["Title", "Price", "Monthly Sales", "Rating", "Date"]
rows = [title, price, monthly_sales, rating, today.strftime("%Y-%m-%d")]

print(type(rows))  # Check to be a list because csv requires a list

with open("AMZ_WebScrape_Dataset.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(rows)
    
#"C:\Users\noaha\AppData\Local\Programs\Microsoft VS Code\AMZ_WebScrape_Dataset.csv"

<class 'list'>


In [8]:
#Seeing what the data will look like in a data frame 

import pandas as pd

df = pd.read_csv(r"AMZ_WebScrape_Dataset.csv")

print(df)


                          Title    Price             Monthly Sales  \
0  PlayStation 5 Console (Slim)  £478.50  1K+ bought in past month   

               Rating        Date  
0  4.7 out of 5 stars  2024-05-19  


Appending data to csv file

In [9]:
with open("AMZ_WebScrape_Dataset.csv", "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(rows)

In [10]:
import requests
from bs4 import BeautifulSoup
import datetime
import csv
import time

def check_price():
    url = "https://www.amazon.co.uk/Playstation-711719577157-PlayStation-Console-Slim/dp/B0CM9VHGY7/ref=sr_1_3?crid=3GOR693VEO69U&dib=eyJ2IjoiMSJ9.24sD8SdnVv-3aB5oHBS0Xg-wNRbkkttjoPJRofu3IBpRHCO0w-06laB77gEy8WiYVhqRP-fVdGes6kC5ZPVABiHVzJCQEFTxrky9NW-blNoVatZ1pPMUCOvgPanUPGxA6jbo63_7GByqIA0MAC75i5_MJ7c22zFi2vk5wsJ6ogGKqXL6zsUjY9ctD3TVpADH1ZcCMTEd4COQJ95Q0AVAQ8R_AqX3BEnxttt_eboVgGc.niIIzjtkhGZjZU7E8TDlZtG88NgF1zJhglL5FUjfQhw&dib_tag=se&keywords=playstation&qid=1716120758&sprefix=playstation%2Caps%2C778&sr=8-3&th=1"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    title = None
    try:
        title_span = soup.find("span", {"id": "productTitle"})
        if title_span:
            title = title_span.text.strip()
    except Exception as e:
        print(f"Error occurred while extracting title: {e}")

    price = None
    try:
        price_span = soup.find("span", {"class": "aok-offscreen"})
        if price_span:
            price = price_span.text.strip()[1:]
    except Exception as e:
        print(f"Error occurred while extracting price: {e}")

    rating = None
    try:
        rating_span = soup.find("span", {"class": "a-icon-alt"})
        if rating_span:
            rating = rating_span.text.strip()
    except Exception as e:
        print(f"Error occurred while extracting rating: {e}")

    monthly_sales = None
    try:
        monthly_sales_span = soup.find("span", {"id": "social-proofing-faceout-title-tk_bought"})
        if monthly_sales_span:
            monthly_sales = monthly_sales_span.text.strip()
    except Exception as e:
        print(f"Error occurred while extracting monthly sales: {e}")
        
    today = datetime.datetime.now()
    print(today.strftime("%Y-%m-%d"))

    rows = [title, price, monthly_sales, rating, today.strftime("%Y-%m-%d %H:%M:%S")]

    with open("AMZ_WebScrape_Dataset.csv", "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(rows)


while(True):
    check_price()
    time.sleep(86_400) #will automatically run this function every 24 hours /Automated if run in the background


2024-05-19
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\noaha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\noaha\AppData\Local\Temp\ipykernel_26840\217756761.py", line 56, in <module>
    time.sleep(5)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\noaha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\noaha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core

In [ ]:
import pandas as pd

df = pd.read_csv("AMZ_WebScrape_Dataset.csv")

print(df)

Allow user to do a search for a specific item and select it , then send out an email to the user with the price of the item when it drops below a certain price.